## Import packages

In [1]:
import os
import pandas as pd

## Path configuration

In [2]:
print("Current working dir:", os.getcwd())
data_dir = "."
raw_dir = os.path.join(data_dir, "raw")
processed_dir = os.path.join(data_dir, "processed")
tempo_dir = os.path.join(data_dir, "tempo")
kidney_dir = os.path.join(data_dir, "kidney")
os.makedirs(processed_dir, exist_ok=True)
os.makedirs(tempo_dir, exist_ok=True)

Current working dir: /blue/qsong1/daling.shi/EHRCare_new/ehr_datasets/mimic-iv


In [3]:
print("raw_dir absolute:", os.path.abspath(raw_dir))
print("processed_dir absolute:", os.path.abspath(processed_dir))
print("tempo_dir absolute:", os.path.abspath(tempo_dir))

raw_dir absolute: /blue/qsong1/daling.shi/EHRCare_new/ehr_datasets/mimic-iv/raw
processed_dir absolute: /blue/qsong1/daling.shi/EHRCare_new/ehr_datasets/mimic-iv/processed
tempo_dir absolute: /blue/qsong1/daling.shi/EHRCare_new/ehr_datasets/mimic-iv/tempo


## Extracting demographic features

In [4]:
# read base tables
patients = pd.read_csv(os.path.join(raw_dir, "patients.csv"))
admissions = pd.read_csv(os.path.join(raw_dir, "admissions.csv"))

In [5]:
# Extract patient basic info
patients = patients.rename(columns={"subject_id": "PatientID"})
patients["Sex"] = patients["gender"].map({"M": 1, "F": 0})  
patients = patients[["PatientID", "anchor_age", "Sex"]].rename(columns={"anchor_age": "Age"})

In [6]:
# Extract admission/discharge information
admissions = admissions.rename(columns={
    "subject_id": "PatientID",
    "hadm_id": "AdmissionID",
    "admittime": "AdmissionTime",
    "dischtime": "DischargeTime"
})
admissions = admissions[["PatientID", "AdmissionID", "AdmissionTime", "DischargeTime", "deathtime"]]

## Combine patients and admissions to calculate Outcome and LOS

In [7]:
df = admissions.merge(patients, on="PatientID", how="left")

# Define Outcome (death = 1, survival = 0), LOS (number of days)
df["Outcome"] = df["deathtime"].notnull().astype(int)
df["LOS"] = (pd.to_datetime(df["DischargeTime"]) - pd.to_datetime(df["AdmissionTime"])).dt.days
df["Readmission"] = 0  # Additional logical flags are required, set to 0 here first

df.to_csv(os.path.join(tempo_dir, "demographics.csv"), index=False)

## Read chartevents

In [8]:
input_file = os.path.join(raw_dir, "chartevents.csv.gz")
output_file = os.path.join(tempo_dir, "chartevents_preprocessed.csv")

# keep selected cols
usecols = ["subject_id", "hadm_id", "charttime", "itemid", "valuenum"]

# itemid map
item_map = {
    220045: "HR",
    220179: "NBPs",
    220180: "NBPd",
    220181: "NBPm",
    220210: "RR",
    223830: "PH",
    220734: "PH (dipstick)",
    220277: "SpO2",
    220621: "Glucose",
    223761: "Temp_F",
    220739: "GCSEye",
    223900: "GCSVerbal",
    223901: "GCSMotor",
    223951: "CRT",
    224639: "Weight",
    226531: "AdWeight",
    226707: "Height",
    223835: "FiO2"
}

chunksize = 8_000_000  # can adjust according to RAM
chunks = []

for i, chunk in enumerate(pd.read_csv(input_file, usecols=usecols, chunksize=chunksize)):
    # filter itemid
    chunk = chunk[chunk["itemid"].isin(item_map.keys())]
    # rename cols
    chunk = chunk.rename(columns={
        "subject_id": "PatientID",
        "hadm_id": "AdmissionID",
        "charttime": "RecordTime"
    })
    # add FeatureName
    chunk["FeatureName"] = chunk["itemid"].map(item_map)
    # only keep cols we need
    chunk = chunk[["PatientID", "AdmissionID", "RecordTime", "FeatureName", "valuenum"]]
    
    chunks.append(chunk)
    print(f"finish {i+1} chunks, with {len(chunk)} rows.")


# concat all chunks
chartevents = pd.concat(chunks, ignore_index=True)

# save to new CSV
chartevents.to_csv(output_file, index=False)
print("finished, save at: ", output_file)


finish 1 chunks, with 1000493 rows.
finish 2 chunks, with 1012937 rows.
finish 3 chunks, with 1016013 rows.
finish 4 chunks, with 1011567 rows.
finish 5 chunks, with 978673 rows.
finish 6 chunks, with 1017810 rows.
finish 7 chunks, with 1043085 rows.
finish 8 chunks, with 984780 rows.
finish 9 chunks, with 990185 rows.
finish 10 chunks, with 999172 rows.
finish 11 chunks, with 983734 rows.
finish 12 chunks, with 1000280 rows.
finish 13 chunks, with 1001891 rows.
finish 14 chunks, with 1031487 rows.
finish 15 chunks, with 996825 rows.
finish 16 chunks, with 995402 rows.
finish 17 chunks, with 995876 rows.
finish 18 chunks, with 1041509 rows.
finish 19 chunks, with 1011804 rows.
finish 20 chunks, with 1019996 rows.
finish 21 chunks, with 1007735 rows.
finish 22 chunks, with 1003573 rows.
finish 23 chunks, with 1022351 rows.
finish 24 chunks, with 1005540 rows.
finish 25 chunks, with 1027877 rows.
finish 26 chunks, with 999267 rows.
finish 27 chunks, with 988284 rows.
finish 28 chunks, wi

## Convert to a pivot table

In [9]:
chartevents = pd.read_csv(os.path.join(tempo_dir, "chartevents_preprocessed.csv"))

# count distinct PatientID
num_patients = chartevents["PatientID"].nunique()
print("Number of distinct patients:", num_patients)

Number of distinct patients: 65366


In [10]:
chartevents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54421959 entries, 0 to 54421958
Data columns (total 5 columns):
 #   Column       Dtype  
---  ------       -----  
 0   PatientID    int64  
 1   AdmissionID  int64  
 2   RecordTime   object 
 3   FeatureName  object 
 4   valuenum     float64
dtypes: float64(1), int64(2), object(2)
memory usage: 2.0+ GB


In [11]:
# convert to pivot table
chartevents = chartevents.pivot_table(
    index=["PatientID", "AdmissionID", "RecordTime"], 
    columns="FeatureName", 
    values="valuenum", 
    aggfunc="mean"
).reset_index()

chartevents.to_csv(os.path.join(tempo_dir, "chartevents_wide.csv"), index=False)

## Merge static features + dynamic features

In [8]:
chartevents = pd.read_csv(os.path.join(tempo_dir, "chartevents_wide.csv"))
# df = pd.read_csv(os.path.join(tempo_dir, "demographics.csv"))

In [12]:
chartevents.head(10)

FeatureName,PatientID,AdmissionID,RecordTime,AdWeight,FiO2,GCSEye,GCSMotor,GCSVerbal,Glucose,HR,Height,NBPd,NBPm,NBPs,PH,PH (dipstick),RR,SpO2,Temp_F,Weight
0,10000032,29079034,2180-07-23 12:36:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000032,29079034,2180-07-23 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.7,NaN
2,10000032,29079034,2180-07-23 14:11:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.0,56.0,84.0,NaN,NaN,NaN,NaN,NaN,NaN
3,10000032,29079034,2180-07-23 14:12:00,NaN,NaN,NaN,NaN,NaN,NaN,91.0,NaN,NaN,NaN,NaN,NaN,NaN,24.0,NaN,NaN,NaN
4,10000032,29079034,2180-07-23 14:13:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.0,NaN,NaN
5,10000032,29079034,2180-07-23 14:22:00,86.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,10000032,29079034,2180-07-23 14:30:00,NaN,NaN,NaN,NaN,NaN,NaN,93.0,NaN,59.0,67.0,95.0,NaN,NaN,21.0,97.0,NaN,NaN
7,10000032,29079034,2180-07-23 14:44:00,86.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,10000032,29079034,2180-07-23 14:45:00,NaN,NaN,4.0,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10000032,29079034,2180-07-23 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,94.0,NaN,56.0,64.0,88.0,NaN,NaN,23.0,97.0,NaN,NaN


In [13]:
df.head()

,PatientID,AdmissionID,AdmissionTime,DischargeTime,deathtime,Age,Sex,Outcome,LOS,Readmission
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,52,0,0,0,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,52,0,0,1,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,52,0,0,1,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,52,0,0,2,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaN,19,0,0,0,0


In [14]:
final_df = chartevents.merge(df, on=["PatientID", "AdmissionID"], how="left")
final_df = final_df.sort_values(["PatientID", "RecordTime"])

# save result
save_path = os.path.join(tempo_dir, "mimic4_timeseries.csv")
final_df.to_csv(save_path, index=False)
print(f"Saved processed EHR data to {save_path}")

Saved processed EHR data to ./tempo/mimic4_timeseries.csv


In [15]:
# check final df result
# check head
print(final_df.head())

# check basic info
print(final_df.info()) 

# check statistics describe (for numeric col)
print(final_df.describe())  

# check num of rows & cols
print(final_df.shape)

# check cols name
print(final_df.columns.tolist())

   PatientID  AdmissionID           RecordTime  AdWeight  FiO2  GCSEye  \
0   10000032     29079034  2180-07-23 12:36:00       NaN   NaN     NaN   
1   10000032     29079034  2180-07-23 14:00:00       NaN   NaN     NaN   
2   10000032     29079034  2180-07-23 14:11:00       NaN   NaN     NaN   
3   10000032     29079034  2180-07-23 14:12:00       NaN   NaN     NaN   
4   10000032     29079034  2180-07-23 14:13:00       NaN   NaN     NaN   

   GCSMotor  GCSVerbal  Glucose    HR  ...  Temp_F  Weight  \
0       NaN        NaN      NaN   NaN  ...     NaN     NaN   
1       NaN        NaN      NaN   NaN  ...    98.7     NaN   
2       NaN        NaN      NaN   NaN  ...     NaN     NaN   
3       NaN        NaN      NaN  91.0  ...     NaN     NaN   
4       NaN        NaN      NaN   NaN  ...     NaN     NaN   

         AdmissionTime        DischargeTime  deathtime  Age  Sex  Outcome  \
0  2180-07-23 12:35:00  2180-07-25 17:55:00        NaN   52    0        0   
1  2180-07-23 12:35:00  2180

In [16]:
# merge data by date
df = pd.read_csv(os.path.join(tempo_dir, "mimic4_timeseries.csv"))
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14395530 entries, 0 to 14395529
Data columns (total 28 columns):
 #   Column         Dtype  
---  ------         -----  
 0   PatientID      int64  
 1   AdmissionID    int64  
 2   RecordTime     object 
 3   AdWeight       float64
 4   FiO2           float64
 5   GCSEye         float64
 6   GCSMotor       float64
 7   GCSVerbal      float64
 8   Glucose        float64
 9   HR             float64
 10  Height         float64
 11  NBPd           float64
 12  NBPm           float64
 13  NBPs           float64
 14  PH             float64
 15  PH (dipstick)  float64
 16  RR             float64
 17  SpO2           float64
 18  Temp_F         float64
 19  Weight         float64
 20  AdmissionTime  object 
 21  DischargeTime  object 
 22  deathtime      object 
 23  Age            int64  
 24  Sex            int64  
 25  Outcome        int64  
 26  LOS            int64  
 27  Readmission    int64  
dtypes: float64(17), int64(7), object(4)
memo

In [17]:
# merge data by date

# only reserve y-m-d precision for `RE_DATE` and `Discharge time` columns
date_columns = ['RecordTime', 'AdmissionTime', 'DischargeTime']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')
    df[col] = df[col].dt.strftime('%Y-%m-%d')

# Merge by PatientID and RecordTime
#df = df.groupby(['PatientID', 'RecordTime', 'AdmissionTime', 'DischargeTime'], dropna=True, as_index = False).mean()
group_cols = ['PatientID', 'RecordTime', 'AdmissionID']
df = (
    df.groupby(group_cols, dropna=True, as_index=False)
      .agg({
          **{col: 'mean' for col in df.select_dtypes(include='number').columns},
          **{col: 'first' for col in df.select_dtypes(exclude='number').columns if col not in group_cols}
      })
)

df.to_csv(os.path.join(tempo_dir, "mimic4_formatted.csv"), index=False)
df.head()

,RecordTime,PatientID,AdmissionID,AdWeight,FiO2,GCSEye,GCSMotor,GCSVerbal,Glucose,HR,...,Temp_F,Weight,Age,Sex,Outcome,LOS,Readmission,AdmissionTime,DischargeTime,deathtime
0,2180-07-23,10000032.0,29079034.0,86.7,NaN,4.000000,6.000000,4.666667,115.0,96.500000,...,98.966667,NaN,52.0,0.0,0.0,2.0,0.0,2180-07-23,2180-07-25,None
1,2150-11-02,10000690.0,25860671.0,121.6,70.0,4.000000,6.000000,5.000000,NaN,71.500000,...,97.700000,NaN,86.0,0.0,0.0,9.0,0.0,2150-11-02,2150-11-12,None
2,2150-11-03,10000690.0,25860671.0,121.6,78.0,3.833333,5.500000,4.333333,77.0,80.521739,...,97.833333,NaN,86.0,0.0,0.0,9.0,0.0,2150-11-02,2150-11-12,None
3,2150-11-04,10000690.0,25860671.0,121.6,57.5,4.000000,5.666667,4.666667,102.0,80.875000,...,97.120000,NaN,86.0,0.0,0.0,9.0,0.0,2150-11-02,2150-11-12,None
4,2150-11-05,10000690.0,25860671.0,NaN,55.0,3.833333,5.500000,4.500000,107.0,95.538462,...,98.300000,NaN,86.0,0.0,0.0,9.0,0.0,2150-11-02,2150-11-12,None


## Clean Outlier

In [18]:
# clean outlier
df = pd.read_csv(os.path.join(tempo_dir, "mimic4_formatted.csv"))

# cols need to be cleaned
cols = ['NBPd', 'FiO2', 'Glucose', 'HR', 'Height', 'NBPm', 'SpO2', 'RR', 'NBPs', 'Temp_F', 'Weight', 'PH',
        'AdWeight', 'GCSEye', 'GCSMotor', 'GCSVerbal','PH (dipstick)']

# process column by column and replace abnormal values ​​with NaN
for col in cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df.loc[(df[col] < lower) | (df[col] > upper), col] = pd.NA

# result
print(df.isna().sum())   # Check how many NaNs there are in each column
null_percent = df.isna().mean() * 100 # check null values percentage
print("Null values percentage per column:\n", null_percent)
df.to_csv(os.path.join(processed_dir, "mimic4_formatted_clean.csv"), index=False)

RecordTime            0
PatientID             0
AdmissionID           0
AdWeight         320586
FiO2             235135
GCSEye            55957
GCSMotor          65516
GCSVerbal         20511
Glucose           77197
HR                17073
Height           388131
NBPd              97706
NBPm              97465
NBPs              96356
PH               293567
PH (dipstick)    382933
RR                22629
SpO2              21987
Temp_F            62778
Weight           293748
Age                   0
Sex                   0
Outcome               0
LOS                   0
Readmission           0
AdmissionTime         0
DischargeTime         0
deathtime        361082
dtype: int64
Null values percentage per column:
 RecordTime        0.000000
PatientID         0.000000
AdmissionID       0.000000
AdWeight         74.331543
FiO2             54.518748
GCSEye           12.974273
GCSMotor         15.190637
GCSVerbal         4.755711
Glucose          17.899010
HR                3.958571
Height   

In [19]:
df = pd.read_csv(os.path.join(processed_dir, "mimic4_formatted_clean.csv"))
num_patients = df["PatientID"].nunique()
print("Number of distinct patients:", num_patients)

Number of distinct patients: 65366


## Filter: Acute kidney failure with tubular necrosis

In [20]:
patient_ids = pd.read_csv(os.path.join(kidney_dir, "kidney_patients_id.csv"))
filtered_df = df[df["PatientID"].isin(patient_ids["PatientID"])]
filtered_df.to_csv(os.path.join(processed_dir, "mimic4_kidney.csv"), index=False)
print(filtered_df.info())
num_patients = filtered_df["PatientID"].nunique()
print("Number of distinct patients:", num_patients)
null_percent = df.isna().mean() * 100 # check null values percentage
print("Null values percentage per column:\n", null_percent)



<class 'pandas.core.frame.DataFrame'>
Index: 89745 entries, 27 to 430949
Data columns (total 28 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   RecordTime     89745 non-null  object 
 1   PatientID      89745 non-null  float64
 2   AdmissionID    89745 non-null  float64
 3   AdWeight       17608 non-null  float64
 4   FiO2           53154 non-null  float64
 5   GCSEye         74999 non-null  float64
 6   GCSMotor       69498 non-null  float64
 7   GCSVerbal      85969 non-null  float64
 8   Glucose        77316 non-null  float64
 9   HR             86269 non-null  float64
 10  Height         6628 non-null   float64
 11  NBPd           61342 non-null  float64
 12  NBPm           61479 non-null  float64
 13  NBPs           61507 non-null  float64
 14  PH             38833 non-null  float64
 15  PH (dipstick)  11386 non-null  float64
 16  RR             84242 non-null  float64
 17  SpO2           85240 non-null  float64
 18  Temp_F   

In [21]:
all_df = pd.read_csv(os.path.join(processed_dir, "mimic4_formatted_clean.csv"))
kidney_df = pd.read_csv(os.path.join(processed_dir, "mimic4_kidney.csv"))

dfs = [all_df, kidney_df]
for df in dfs:
    num_patients = df["PatientID"].nunique()
    print("Number of distinct patients:", num_patients)
    patient_outcome = df.groupby('PatientID')['Outcome'].first().reset_index()
    outcome = patient_outcome['Outcome'].value_counts()
    print("Dead(1) & Survival(0):\n", outcome)
    balance_percent = patient_outcome['Outcome'].value_counts(normalize=True) * 100
    print(balance_percent)

Number of distinct patients: 65366
Dead(1) & Survival(0):
 Outcome
0.0    58286
1.0     7080
Name: count, dtype: int64
Outcome
0.0    89.168681
1.0    10.831319
Name: proportion, dtype: float64
Number of distinct patients: 6165
Dead(1) & Survival(0):
 Outcome
0.0    4619
1.0    1546
Name: count, dtype: int64
Outcome
0.0    74.922952
1.0    25.077048
Name: proportion, dtype: float64
